### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
31,42701291,31717,79085,51530,0,0,1145,5,2025-06-19 19:30,2025-06-19 19:30,5,2,3514,4,1
32,42694285,123111,79085,1001,2,78,1145,6,2025-06-20 21:59,2025-06-20 21:59,2,1,9664,6,1
33,42937084,79085,149122,51530,0,0,1146,0,2025-06-21 12:30,2025-06-21 12:30,1,4,1779,3,1
34,42740222,79866,79085,1349,1,71,1146,1,2025-06-22 21:59,2025-06-22 21:59,0,0,13147,5,1
35,42947126,22807,79085,51530,0,0,1146,2,2025-06-23 18:30,2025-06-23 18:30,4,0,8206,3,1
36,42947132,1994,79085,51530,0,0,1146,3,2025-06-24 19:35,2025-06-24 19:35,2,0,6493,0,1
37,42740232,79085,122169,1349,2,71,1146,4,2025-06-25 21:59,2025-06-25 21:59,1,5,9483,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,31717,37669,31489,0,11,3,1,0,4-3-3-,11,10,8,42701291
1,79085,32306,38486,0,15,4,1,1,4-4-2b,9,10,11,42701291
0,123111,37799,51203,0,8,2,1,0,433,8,7,9,42694285
1,79085,55482,42078,1,2,7,1,0,4-4-2c,6,8,8,42694285
0,79085,41271,32613,2,2,0,0,0,4-3-3a,8,9,11,42937084
1,149122,28823,37481,0,12,1,0,0,VITEAM_OFF,10,9,11,42937084
0,79866,40888,33813,0,3,2,0,0,Haikyuu!,8,8,9,42740222
1,79085,29418,36493,2,9,3,1,0,4-3-3aRU,10,10,11,42740222
0,22807,27335,32132,7,11,1,0,0,AZERBAYCAN,9,9,9,42947126
1,79085,42718,37921,1,5,2,1,0,4-2-3-1a,10,10,10,42947126


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1822

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,78751,2,6,2,0,0,4,1,6503004002002,1349
1,78671,2,6,2,0,0,6,4,6502006002007,1349
2,23775,2,4,1,1,0,5,1,4504005001011,1349
3,6276,2,4,1,1,0,3,0,4503003001012,1349
4,79587,2,3,1,0,1,11,1,3510011001001,1349
5,89569,2,3,1,0,1,8,3,3505008001004,1349
6,122169,2,3,1,0,1,8,5,3503008001008,1349
7,79866,2,1,0,1,1,1,2,1499001000005,1349
8,124139,2,1,0,1,1,1,4,1497001000010,1349
9,23755,2,1,0,1,1,2,6,1496002000009,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()